<a href="https://colab.research.google.com/github/shiffa-04/LangChain/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4

In [ ]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.5 MB/s eta 0:00:00


In [ ]:
pip install pymupdf


In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
loader = PyMuPDFLoader("story.pdf")
documents = loader.load()
print(documents)

[Document(metadata={'source': 'story.pdf', 'file_path': 'story.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20240827175423Z', 'modDate': '', 'trapped': ''}, page_content='Kabir, Aryan, Nick, Hamza, and Zayn embarked on their long-awaited trekking adventure early in the morning, the excitement palpable as they \nleft the city behind and ventured into the heart of nature. Zayn, the reliable leader, was in his element as he took charge, studying the map and \nensuring everyone was geared up for the journey. His calm demeanor and clear direction reassured everyone that they were in safe hands. Nick, \nthe group’s perpetual jokester, wasted no time in lifting everyone’s spirits, turning even the initial uphill climb into a comedy show with his witty \nremarks and light-hearted banter. Every stumble became an opportunity for a joke, every pause a chance to make

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

19


In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:")
    print(chunk)
    print("-" * 40)

Chunk 1:
page_content='Kabir, Aryan, Nick, Hamza, and Zayn embarked on their long-awaited trekking adventure early in the morning, the excitement palpable as they 
left the city behind and ventured into the heart of nature. Zayn, the reliable leader, was in his element as he took charge, studying the map and 
ensuring everyone was geared up for the journey. His calm demeanor and clear direction reassured everyone that they were in safe hands. Nick,' metadata={'source': 'story.pdf', 'file_path': 'story.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20240827175423Z', 'modDate': '', 'trapped': ''}
----------------------------------------
Chunk 2:
page_content='ensuring everyone was geared up for the journey. His calm demeanor and clear direction reassured everyone that they were in safe hands. Nick, 
the group’s perpetual jokester, wasted no time in lifting ev

In [ ]:
vectorStores = Chroma.from_documents(documents = chunks, embedding = OpenAIEmbeddings())

In [ ]:
retriever = vectorStores.as_retriever()

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use two sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
result = rag_chain.invoke({"input":"who is Kabir"})
print(result["answer"])

Kabir is described as a knowledgeable individual who enriched a hiking journey by offering insights about various topics encountered during the trip and sharing stories related to nature and legends.


In [ ]:
result = rag_chain.invoke({"input":"Who was the funny character"})
print(result["answer"])

Nick was the funny character who lifted everyone's spirits with witty remarks and light-hearted banter during the trek.


In [ ]:
result = rag_chain.invoke({"input":"what happened during the rain"})
print(result["answer"])

Despite the rain starting to fall, the group found joy in the situation and engaged in a spontaneous celebration, including dancing and a friendly competition of catching raindrops. Hamza also captured the moment with his camera, turning it into a beautiful impromptu photoshoot.


In [ ]:
result = rag_chain.invoke({"input":"Describe Zayn's character"})
print(result["answer"])

Zayn is described as a reliable leader who is calm, clear in his direction, and ensures everyone is prepared for the journey. He takes charge, studies the map, and reassures others that they are in safe hands.


**ADDING CHAT HISTORY**

In [ ]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Who is hamza?"},
    config={
        "configurable": {"session_id": "abc"}
    },
)["answer"]

'Hamza is the dedicated photographer mentioned in the context, who lagged behind the group a bit during the trek not because he was slow, but because he was focused on capturing every beautiful moment with his camera. His passion for photography was evident as he documented the trek and the breathtaking scenes they encountered.'

In [ ]:
conversational_rag_chain.invoke(
    {"input": "what did he do when it started raining?"},
    config={"configurable": {"session_id": "abc"}},
)["answer"]

'When it started raining, Hamza captured the rain-soaked fun with his camera, turning the spontaneous celebration into a beautiful, impromptu photoshoot. Despite the drizzle, he made sure to preserve the laughter and joy of the moment through his photography.'

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Who was the leader of the group during the trek?"},
    config={
        "configurable": {"session_id": "abcd"}
    },
)["answer"]

'Zayn was the leader of the group during the trek. He took charge, studied the map, and ensured everyone was geared up for the journey, reassuring everyone with his calm demeanor and clear direction.'

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What leadership qualities did he display during the trek?"},
    config={
        "configurable": {"session_id": "abcd"}
    },
)["answer"]

'Zayn displayed leadership qualities such as taking charge, studying the map, ensuring everyone was prepared for the journey, maintaining a calm demeanor, providing clear direction, and reassuring everyone that they were in safe hands.'